In [ ]:
import pandas as pd
import json
import datetime
import time
import os
import numpy as np
import lightgbm as lgb
from config import INTERVENTION_CALENDAR, DATA_CONSUMPTION_RAW_FOLDER, DATA_METADATA_PROCESSED_FILE, DATA_HOLIDAYS_PROCESSED_FILE, DATA_CONSUMPTION_PROCESSED_FILE,DATA_METADATA_PROCESSED_FILE
from config import DATA_CONSUMPTION_PROCESSED_FILE,DATA_CONSUMPTION_SEMI_PROCESSED_FILE,  DATA_WEATHER_PROCESSED_FILE,  DATA_METADATA_PROCESSED_FILE, DATA_HOLIDAYS_PROCESSED_FILE, DATA_ISO_CONSUMPTION_PROCESSED_FILE, DATA_ENTHALPY_GRADIENTS_PROCESSED_FILE, DATA_SOLAR_GAINS_PROCESSED_FILE, DATA_GBM_CONSUMPTION_PROCESSED_FILE
from lgbm_imputer import imputer
import matplotlib
import warnings
from auxiliary import week_of_month
warnings.filterwarnings('ignore')

In [ ]:
interventions = INTERVENTION_CALENDAR

In [ ]:
#join to metadata
metadata_df = pd.read_excel(DATA_METADATA_PROCESSED_FILE, sheets='SENSORS')

In [ ]:
meta_data_faulty = pd.read_excel(DATA_METADATA_PROCESSED_FILE, sheet_name="FAULTY")

In [ ]:
def f(x):
    return time.asctime(time.localtime(x/1000))

In [ ]:
headdir1 = os.path.join(DATA_CONSUMPTION_RAW_FOLDER, 'DAILY_ntufrs1')
headdir2 = os.path.join(DATA_CONSUMPTION_RAW_FOLDER, 'DAILY_ntufrs2')
headdir3 = os.path.join(DATA_CONSUMPTION_RAW_FOLDER, 'DAILY_ntufrs3')
headdirs = [headdir1,headdir2, headdir3]
data_holidays = DATA_HOLIDAYS_PROCESSED_FILE
data_output_path = DATA_CONSUMPTION_PROCESSED_FILE

In [ ]:
#get all the data that exists
data_raw = pd.DataFrame()
for headdir in headdirs:
    list_data_files = os.listdir(headdir)
    for file in list_data_files:
        path_to_data = os.path.join(headdir,file)
        with open(path_to_data) as json_file:
            data = json.load(json_file)
            smapees_in_data = list(data.keys())
            for smapee in smapees_in_data:
                try:
                    data_clean = pd.DataFrame(data[smapee]['consumptions'])[['timestamp', 'consumption']]
                    data_clean['smapee'] = "ID"+str(int(smapee))
                    data_clean['CONSUMPTION_kWh'] = data_clean['consumption']/1000.0
                    data_clean = data_clean.drop('consumption', axis=1)
                except Exception:
                    continue
                data_raw = data_raw.append(data_clean, ignore_index=True)
data_raw['timestamp'] = pd.to_datetime(data_raw['timestamp'].apply(lambda x: f(x)))
data_raw = data_raw.drop_duplicates(['smapee','timestamp'])
data_raw.reset_index(inplace=True, drop=True)
data_raw.smapee.count(), len(data_raw.smapee.unique())

In [ ]:
# smapee = 'ID27968'
# data = data_raw.set_index('timestamp')
# c = data[data['smapee']==smapee]
# length = c.shape[0]
# miss = c[['CONSUMPTION_kWh']].isnull().sum()*100/length
# ax = c[['CONSUMPTION_kWh']].plot()
# ax.legend([str(smapee)+' '+str(miss)]);

In [ ]:
# data = data_raw.set_index('timestamp')
# trend = data['smapee'].unique()
# for smapee in trend:
#     c = data[data['smapee']==smapee]
#     length = c.shape[0]
#     miss = c[['CONSUMPTION_kWh']].isnull().sum()*100/length
#     ax = c[['CONSUMPTION_kWh']].plot()
#     ax.legend([str(smapee)+' '+str(miss)]);

In [ ]:
#include all the dates that are missing (for now we do from the first date of intervention to the latest one)
unique_smapees = data_raw['smapee'].unique()
data_raw_full_extent = data_raw.set_index(['smapee', 'timestamp'], drop=False).sort_index()

#range of all the thee experiments, min and max
intervention_data = interventions[1]
earliest_date = '2018-01-01 00:00'
intervention_data = interventions[3]
latest_date = '2019-12-31 23:00'
range_experiments = pd.date_range(start=earliest_date, end=latest_date, freq='D')

# construct full index w/o missing dates
full_index = pd.MultiIndex.from_product([unique_smapees, range_experiments])
data_raw_full_extent = data_raw_full_extent.reindex(full_index)
data_raw_full_extent['smapee'] = data_raw_full_extent.index.get_level_values(0)
data_raw_full_extent['timestamp'] = data_raw_full_extent.index.get_level_values(1)
data_raw_full_extent.reset_index(inplace=True, drop=True)
data_raw_full_extent.smapee.count(), len(data_raw_full_extent.smapee.unique())

In [ ]:
#erase faulty and fill in missing dates
unique_smapees = data_raw_full_extent['smapee'].unique()
data_raw_full_extent_after_faulty = data_raw_full_extent.copy()
for smapee in unique_smapees:
    faulty = meta_data_faulty[meta_data_faulty['smapee']==smapee]
    if faulty.empty == False:
        #print("filling range of data in sensor with null", smapee)
        range_na = pd.date_range(start=faulty['invalid from'].values[0], end=faulty['invalid to'].values[0], freq='D')
        data_raw_full_extent_after_faulty.loc[(data_raw_full_extent_after_faulty['timestamp'].isin(range_na))&(data_raw_full_extent_after_faulty['smapee']==smapee), 'CONSUMPTION_kWh'] = np.nan
        
        #try the second range
        if pd.isnull(faulty['2invalid from'].values[0]) == False:  
            #print("filling range of data in sensor with null", smapee)
            range_na = pd.date_range(start=faulty['2invalid from'].values[0], end=faulty['2invalid to'].values[0], freq='D')
            data_raw_full_extent_after_faulty.loc[(data_raw_full_extent_after_faulty['timestamp'].isin(range_na))&(data_raw_full_extent_after_faulty['smapee']==smapee), 'CONSUMPTION_kWh'] = np.nan
    
    #if explicityl we say to discard these values
    if faulty['DSICARD'].values == True:
        print("discarding sensor because it is corrupted", smapee)
        data_raw_full_extent_after_faulty = data_raw_full_extent_after_faulty[data_raw_full_extent_after_faulty['smapee'] != smapee]
data_raw_full_extent_after_faulty.reset_index(inplace=True, drop=True)
len(data_raw_full_extent_after_faulty.smapee.unique())

In [ ]:
c = data_raw_full_extent_after_faulty.set_index('timestamp')
c = c[c['smapee']=="ID28099"]
length = c.shape[0]
miss = c[['CONSUMPTION_kWh']].isnull().sum()*100/length
ax = c[['CONSUMPTION_kWh']].plot()
ax.legend([str(28099)+' '+str(miss)]);

In [ ]:
#number of smapees despues de eliminar faulty
unique_smapees = data_raw_full_extent_after_faulty['smapee'].unique()
len(unique_smapees)

In [ ]:
#erase outliers (99th percentile)
quantile = data_raw_full_extent_after_faulty.groupby('smapee').quantile(0.99)
for smapee in unique_smapees:
    thpercentile = quantile.loc[smapee,'CONSUMPTION_kWh']
    consumption = data_raw_full_extent_after_faulty.loc[data_raw_full_extent_after_faulty['smapee']==smapee,'CONSUMPTION_kWh'].values
    data_raw_full_extent_after_faulty.loc[data_raw_full_extent_after_faulty['smapee']==smapee,'CONSUMPTION_kWh'] = [np.nan if x >= thpercentile else x for x in consumption] 

In [ ]:
data = data_raw_full_extent_after_faulty.set_index('timestamp')
for smapee in unique_smapees:
    c = data[data['smapee']==smapee]
    length = c.shape[0]
    miss = c[['CONSUMPTION_kWh']].isnull().sum()*100/length
    ax = c[['CONSUMPTION_kWh']].plot()
    ax.legend([str(smapee)+' '+str(miss)]);

In [ ]:
#now get flags to know what sensors can be used for the experiments
interventions = INTERVENTION_CALENDAR
clean_data = data_raw_full_extent_after_faulty.set_index('timestamp')

#create empty 
metadata_df['VALID_FOR_BOTH'] = False
metadata_df['VALID_FOR_PRE'] = False
metadata_df['VALID_FOR_POST'] = False

#threshold to reject
threshold_pre = 0.65
threshold_postperiod = 0.85

#iterate over metadata:
lenght = metadata_df.shape[0]
for record in range(lenght):
    #get what record to analyze
    smapee = metadata_df.loc[record,'smapee']
    experiment = metadata_df.loc[record,'EXPERIMENT']
    smapee_tested = clean_data[clean_data['smapee']==smapee]
    
    #check if it complies, i.e., is there sufficient data for the post-period
    intervention_data = interventions[experiment]
    
    pre_period = intervention_data[1]
    range_pre_intervention_period = pd.date_range(start=pre_period[0], end=pre_period[1], freq='D')
    records_all_pre_period = len(range_pre_intervention_period)
    records_it_has_pre = len(smapee_tested[(smapee_tested.index.isin(range_pre_intervention_period))&(smapee_tested['CONSUMPTION_kWh']>0.0)])
    perc_it_has_pre = records_it_has_pre /records_all_pre_period
    
    post_period = intervention_data[2]
    range_post_intervention_period = pd.date_range(start=post_period[0], end=post_period[1], freq='D')
    records_all_post_period = len(range_post_intervention_period)
    records_it_has_post = len(smapee_tested[(smapee_tested.index.isin(range_post_intervention_period))&(smapee_tested['CONSUMPTION_kWh']>0.0)])
    perc_it_has_post = records_it_has_post /records_all_post_period
    
    
    if perc_it_has_post > threshold_postperiod:
        metadata_df.loc[record,'VALID_FOR_POST'] = True
        #print("this smpaee {} has {} % of the data in postperiod of in experiment {} and {} % data in preperiod".format(smapee, perc_it_has_post*100, experiment, perc_it_has_pre*100))
    
    if perc_it_has_pre > threshold_pre:
        metadata_df.loc[record,'VALID_FOR_PRE'] = True
        #print("this smpaee {} has {} % of the data in postperiod of in experiment {} and {} % data in preperiod".format(smapee, perc_it_has_post*100, experiment, perc_it_has_pre*100))
    
    if metadata_df.loc[record,'VALID_FOR_PRE'] & metadata_df.loc[record,'VALID_FOR_POST']:
        metadata_df.loc[record,'VALID_FOR_BOTH'] = True
    
#print Valid:
def get_number_valid_per_experiment(metadata_df, experiment, column_name):
    exp1 = metadata_df[metadata_df['EXPERIMENT'] == experiment]
    num_tot = exp1.shape[0]
    num_valid = (exp1[column_name]==True).sum()
    perc = num_valid*100/num_tot
    print("the number of valid for exp. {} is {} out of {} or {}%".format(experiment, num_valid, num_tot,perc))

print("HAS DATA FOR THE PRE-INTERVENTION PERIOD")
get_number_valid_per_experiment(metadata_df, 1, 'VALID_FOR_PRE')
get_number_valid_per_experiment(metadata_df, 2, 'VALID_FOR_PRE')
get_number_valid_per_experiment(metadata_df, 3, 'VALID_FOR_PRE')

print("HAS DATA FOR THE INTERVENTION AND POSTINTERVENTION PERIOD")
get_number_valid_per_experiment(metadata_df, 1, 'VALID_FOR_POST')
get_number_valid_per_experiment(metadata_df, 2, 'VALID_FOR_POST')
get_number_valid_per_experiment(metadata_df, 3, 'VALID_FOR_POST')

print("HAS DATA FOR THE BOTH PERIODS PERIOD")
get_number_valid_per_experiment(metadata_df, 1, 'VALID_FOR_BOTH')
get_number_valid_per_experiment(metadata_df, 2, 'VALID_FOR_BOTH')
get_number_valid_per_experiment(metadata_df, 3, 'VALID_FOR_BOTH')

In [ ]:
data_raw_full_extent_after_faulty

In [ ]:
clean_data

In [ ]:
#LETS START WITH THE IMPUTATION WHERE VALUES ARE VALID FOR BOTH
holidays_df = pd.read_csv(DATA_HOLIDAYS_PROCESSED_FILE)
holidays_df['timestamp'] = pd.to_datetime(holidays_df['timestamp'])
gradients_df = pd.read_csv(DATA_ENTHALPY_GRADIENTS_PROCESSED_FILE)
gradients_df['timestamp'] = pd.to_datetime(gradients_df['timestamp'])

#make merge
data_very_clean = data_raw_full_extent_after_faulty.merge(metadata_df, left_on='smapee', right_on='smapee')
# data_very_clean = data_very_clean.merge(holidays_df, left_on='timestamp', right_on='timestamp', how='left')
# data_very_clean = data_very_clean.merge(gradients_df, left_on='timestamp', right_on='timestamp', how='left')
data_very_clean

In [ ]:
#valid
data_very_clean[data_very_clean['VALID_FOR_BOTH']==True]

In [ ]:
%load_ext autoreload
%autoreload 2
def model_imputer(data_train):
    # lgb needs values that start in 0 if they are categorical
    data_train['year'] = np.array(data_train['timestamp'].dt.year, dtype=np.uint16)
    data_train['month'] = np.array(data_train['timestamp'].dt.month, dtype=np.uint8) - 1
    data_train['dayofweek'] = np.array(data_train['timestamp'].dt.dayofweek, dtype=np.uint8)
    data_train['dayofyear'] = np.array(data_train['timestamp'].dt.dayofyear, dtype=np.uint16) - 1
    data_train['weekofyear'] = np.array(data_train['timestamp'].dt.weekofyear, dtype=np.uint8) - 1
    data_train['weekday'] = data_train['dayofweek'].apply(lambda x: 1 if 0<=x<5 else 0)
    data_train['calendar_wom'] = data_train['timestamp'].apply(week_of_month)
    
    target_feature_name = 'CONSUMPTION_kWh'
    numerical_features_list = ['DEG_C_kJperKg',
                               'DEG_DEHUM_kJperKg']
    categorical_features_list = ['smapee',
                                 'dayofweek',
                                 'weekday',
                                 'dayofyear',
                                 'calendar_wom',
                                 'teaching_time',
                                 'school_holiday',
                                 'holiday',
                                 'month']
    
    for c in categorical_features_list:
        data_train[c] = data_train[c].astype('category')
        
    id_column = 'smapee'
    get_best_parameters = False
    window = 7
    params = {'learning_rate': 0.1,
              'num_leaves': 31,
              'max_depth':-1,
              'min_data_in_leaf': 20,
              'num_iterations': 10000,
              'objective': 'rmse',
              'metric': 'rmse'}
    restored = imputer(df=data_train,
                       timestamp_feature_name='timestamp',
                       target_feature_name = target_feature_name,
                       numerical_features_list = numerical_features_list,
                       categorical_features_list = categorical_features_list,
                       id_column = id_column,
                       window = window,
                       get_best_parameters = get_best_parameters,
                       params=params)
    return restored

In [ ]:
# experiment = 1
# result_imputer = []
# for experiment in [1,2,3]:
#     #create training data to also impute
#     intervention_data = interventions[experiment]
#     pre_period = intervention_data[1]
#     post_period = intervention_data[2]
#     period = pd.date_range(start=pre_period[0], end=post_period[1], freq='D')
    
#     # get it from veryclean when all is valid
#     data_train = data_very_clean[data_very_clean['VALID_FOR_BOTH']==True]
#     data_train = data_train[data_train['timestamp'].isin(period)]
#     data_train = data_train[data_train['EXPERIMENT']==experiment]
#     data_train.reset_index(inplace=True, drop=True)

#     #train
#     result_imputer.append(model_imputer(data_train))

data_train = data_very_clean[data_very_clean['VALID_FOR_BOTH']==True]
result_imputer = model_imputer(data_train)

In [ ]:
#save this for further use in the monthly values
output_path = DATA_CONSUMPTION_SEMI_PROCESSED_FILE
result_imputer[['timestamp', 'CONSUMPTION_kWh', 'smapee', 'INTERVENTION', 'ID_CEA']].to_csv(output_path, index=False)

In [ ]:
#number of smapees
for experiment in [1,2,3]:
    #create training data to also impute
    intervention_data = interventions[experiment]
    pre_period = intervention_data[1]
    post_period = intervention_data[2]
    period = pd.date_range(start=pre_period[0], end=post_period[1], freq='D')
    
    if experiment == 1:
        # get it from veryclean when all is valid
        data_experiment1 = result_imputer[result_imputer['timestamp'].isin(period)]
        data_experiment1 = data_experiment1[data_experiment1['EXPERIMENT']==experiment]
        data_experiment1.reset_index(inplace=True, drop=True)
    elif experiment == 2:
        # get it from veryclean when all is valid
        data_experiment2 = result_imputer[result_imputer['timestamp'].isin(period)]
        data_experiment2 = data_experiment2[data_experiment2['EXPERIMENT']==experiment]
        data_experiment2.reset_index(inplace=True, drop=True)
    else:
        # get it from veryclean when all is valid
        data_experiment3 = result_imputer[result_imputer['timestamp'].isin(period)]
        data_experiment3 = data_experiment3[data_experiment3['EXPERIMENT']==experiment]
        data_experiment3.reset_index(inplace=True, drop=True)

In [ ]:
visual = data_experiment1.set_index('timestamp')
visual = visual[visual['smapee']=="ID28088"]
visual = visual[['CONSUMPTION_kWh', 'CONSUMPTION_kWh_all_imputed', 'CONSUMPTION_kWh_imputed']]
font = {'family' : 'Arial',
        'size'   : 18}
ax = visual.plot()
ax.set_xlabel("")
ax.set_ylabel("Daily Consumption [kWh]")
ax.legend(["Measured", "Imputed"]);
matplotlib.rc('font', **font)

In [ ]:
visual = data_experiment3.set_index('timestamp')
visual = visual[visual['smapee']=="ID27785"]
visual = visual[['CONSUMPTION_kWh', 'CONSUMPTION_kWh_all_imputed', 'CONSUMPTION_kWh_imputed']]
font = {'family' : 'Arial',
        'size'   : 18}
ax = visual.plot()
ax.set_xlabel("")
ax.set_ylabel("Daily Consumption [kWh]")
ax.legend(["Measured", "Imputed"]);
matplotlib.rc('font', **font)

In [ ]:
data_experiment1.loc[data_experiment1['CONSUMPTION_kWh'] < 0.0]

In [ ]:
data_experiment2.loc[data_experiment2['CONSUMPTION_kWh'] < 0.0]

In [ ]:
data_experiment3.loc[data_experiment3['CONSUMPTION_kWh'] < 0.0]

In [ ]:
data = data_experiment1.set_index('timestamp')
smapees = data['smapee'].unique()
for smapee in smapees:
    c = data[data['smapee']==smapee]
    length = c.shape[0]
    miss = c[['CONSUMPTION_kWh']].isnull().sum()*100/length
    ax = c[['CONSUMPTION_kWh','CONSUMPTION_kWh_imputed']].plot()
    ax.legend([str(smapee)+' '+str(miss)]);

In [ ]:
data = data_experiment2.set_index('timestamp')
smapees = data['smapee'].unique()
for smapee in smapees:
    c = data[data['smapee']==smapee]
    length = c.shape[0]
    miss = c[['CONSUMPTION_kWh']].isnull().sum()*100/length
    ax = c[['CONSUMPTION_kWh','CONSUMPTION_kWh_imputed']].plot()
    ax.legend([str(smapee)+' '+str(miss)]);

In [ ]:
data = data_experiment3.set_index('timestamp')
smapees = data['smapee'].unique()
for smapee in smapees:
    c = data[data['smapee']==smapee]
    length = c.shape[0]
    miss = c[['CONSUMPTION_kWh']].isnull().sum()*100/length
    ax = c[['CONSUMPTION_kWh','CONSUMPTION_kWh_imputed']].plot()
    ax.legend([str(smapee)+' '+str(miss)]);

In [ ]:
#GATHER ALL RESULTS INTO ONE DATAFRAME
restored = pd.DataFrame()
result_imputer2 = [data_experiment1, data_experiment2, data_experiment3]
for data_experiment in result_imputer2:
    restored = restored.append(data_experiment, ignore_index=True)

def intersection(lst1, lst2): 
    return list(set(lst1) & set(lst2)) 

# #CEHCK THAT VALUES IN EXPERIMENT 1 and 3 are all the same smappees
final = data_experiment2 #(which is clean)
for treatment in ['T1', 'T2', 'T3', 'CONTROL']:
    print(treatment)
    lst1 = list(restored[(restored['EXPERIMENT']==1) & (restored['TREATMENT']== treatment)]['smapee'].unique())
    lst2 = list(restored[(restored['EXPERIMENT']==3) & (restored['TREATMENT']== treatment)]['smapee'].unique())
    intersect =  intersection(lst1, lst2)
    print(len(intersect))
    fin1 = restored[(restored['EXPERIMENT'] == 1) & (restored['TREATMENT']== treatment) & (restored['smapee'].isin(intersect))]
    fin2 = restored[(restored['EXPERIMENT'] == 3) & (restored['TREATMENT']== treatment) & (restored['smapee'].isin(intersect))]
    
    final = final.append(fin1, ignore_index=True)
    final = final.append(fin2, ignore_index=True)
    
final[['timestamp', 'CONSUMPTION_kWh', 'smapee', 'INTERVENTION']].to_csv(data_output_path, index=False)

In [ ]:
final[final["TREATMENT"]=="CONTROL"].smapee.unique()

In [ ]:
# GET COUNT OF IMPUTED
for data_experiment in result_imputer2:
    valid = len(data_experiment['smapee'].unique())
    no_imputed = data_experiment['CONSUMPTION_kWh_imputed'].isnull().sum()
    total = data_experiment.shape[0]
    perc_imputed = (total - no_imputed)*100/total
    print(perc_imputed, valid)

In [ ]:
len(list(data_experiment1[(data_experiment1['EXPERIMENT']==1) & (restored['TREATMENT']== 'T1')]['smapee'].unique()))